In [20]:
!pip install requests
!pip install autopep8
!pip install  urllib
!pip install  streamlit

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [21]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import pandas as pd
import altair as alt
import streamlit as st
import base64

In [22]:
#Auth dashbord https://developer.spotify.com/dashboard/applications
#Env value setting => open jupyter  https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html
# client_id = os.environ.get("CLIENTID")
# client_secret=os.environ.get("CLIENTSEC")
client_id ="7ec0b77501cf4109a5f2785a3006341d"
client_secret="af4ae97c8eac47688177a2f4b3a59a41"

In [23]:
client_creds=f"{client_id}:{client_secret}"
type(client_creds)

str

In [24]:
#VALUE: Basic <base64 encoded client_id:client_secret>"
import base64
client_creds=f"{client_id}:{client_secret}"
print(type(client_creds))
     
client_creds_b64=base64.b64encode(client_creds.encode()) 
client_creds_b64

<class 'str'>


b'N2VjMGI3NzUwMWNmNDEwOWE1ZjI3ODVhMzAwNjM0MWQ6YWY0YWU5N2M4ZWFjNDc2ODgxNzdhMmY0YjNhNTlhNDE='

In [25]:
#Request Access Token
#Flow https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
token_url='https://accounts.spotify.com/api/token'
method="POST"

token_data={
    "grant_type": 'client_credentials'
}

# request must include the following HTTP headers:,Value
#header_parameter

token_headers={
    "Authorization": f"Basic {client_creds_b64.decode()}"#Basic <base64 encoded client_id:client_secret>"
}


In [26]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)


{'access_token': 'BQA632RlGo9HHkdoZCp8Me--n4-5szBprBxCq_jMIC0jpjD-sP9j6ZMbWKjGR8XSD4GvDFad-X2PU-RDFGo', 'token_type': 'Bearer', 'expires_in': 3600}


In [27]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)

if valid_request:
    token_response_data=r.json()
    now=datetime.datetime.now()
    access_token=token_response_data["access_token"]
    exprires_in=token_response_data['expires_in']
    expires=now+datetime.timedelta(seconds=exprires_in)
    didi_expires=expires < now



{'access_token': 'BQCNOwzEnv1M4XdIdSJxmmYNnoHc8UMUAMEPtW9pKKAMKJ5Nzqdg8IY58HnO-a-Ekm6ZLKuXyEzuT1dgEDw', 'token_type': 'Bearer', 'expires_in': 3600}


In [28]:
#class
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
                return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [29]:
spotify=SpotifyAPI(client_id,client_secret)

In [30]:
spotify.perform_auth()

True

In [31]:
access_token=spotify.access_token

In [48]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q": "kill bill", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=kill+bill&type=track
https://api.spotify.com/v1/search?q=kill+bill&type=track
200


In [49]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=kill+bill&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3KDKJ7Z0kgJrQS1CH43PM4'},
       'href': 'https://api.spotify.com/v1/artists/3KDKJ7Z0kgJrQS1CH43PM4',
       'id': '3KDKJ7Z0kgJrQS1CH43PM4',
       'name': 'Hatesonny',
       'type': 'artist',
       'uri': 'spotify:artist:3KDKJ7Z0kgJrQS1CH43PM4'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      '

In [514]:
#class+serch method
#another search method type https://developer.spotify.com/documentation/web-api/reference/#/operations/search

class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
            raise Exception("Not authenticate client")
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
                
        token=self.access_token
        expires=self.access_token_expires
        now=datetime.datetime.now()
        
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token==None:
            self.perform_auth()
            return self.get_access_token
            
        return token
    
    #header
    def get_resource_header(self):
            access_token = self.get_access_token()
            headers = {
                "Authorization": f"Bearer {access_token}"
            }
            return headers

    #depending on input data, get Json
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    #alubum json
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    #artist json 
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')    

    #research Json
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    #query : dic or string  %20 is space
    def search(self,query=None, operator=None,operator_query=None ,search_type='album'):
        if query == None:
               raise Exception("A query is requred")
               
        if isinstance(query,dict):
               query=" ".join([f"{key},{value}"for key,value in query.items()])#inline 
                
        if operator != None and operator_query!=None:
            operator=operator.upper()
            if operator== "or" or operator.lower=="not":
                operator=operator.upper()
                if isinstance(operator_query,str):
                    query=f"{query}{operator}{operator_query}"
               
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    
    #playlist_json_data
    def playlist_search_json_createdata(self,query):
        playlists_json=self.search(query=query,search_type="playlist")

        #Data
        need_playlist= []
        for i, item in enumerate(playlists_json["playlists"]["items"]):
            need_playlist.append((i,
                          item["name"], 
                          item["external_urls"]["spotify"],
                          item["id"],
                          item["images"][0]["url"],
                         
         ))
            playlist_df = pd.DataFrame(need_playlist,index=None,columns=('item','Name','PlaylistURL','ID','ImageURL'))
        return playlist_df

    #get top one playlist tracks json [0]
    def get_playlist_item_json(self,one_playlist_id): #playlist_df[0]
        headers = self.get_resource_header()
        endpoint = f"https://api.spotify.com/v1/playlists/{one_playlist_id}/tracks"
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
# trackdata_organised
    
    def spotify_get_organised_data(self,Name_of_Movie):
        #Data
        Data = self.search({"album": f"{Name_of_Movie}"}, search_type="track")
        
        #need contents 
        need = []
        for i, item in enumerate(Data['tracks']['items']):
            track = item['album']
            track_id = item['id']
            song_name = item['name']
            popularity = item['popularity']
            need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))

        #data frame top10
        Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity',"albumID"))

        access_token = self.access_token
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/audio-features/"

        #Full data
        Feat_df = pd.DataFrame()
        for id in Track_df['Id'].iteritems():
            track_id = id[1]
            lookup_url = f"{endpoint}{track_id}"
            ra = requests.get(lookup_url, headers=headers)
            audio_feat = ra.json()
            Features_df = pd.DataFrame(audio_feat, index=[0])
            Feat_df = Feat_df.append(Features_df)
        Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")

        #sort
        Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)

        #chart df 
        chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
        Name_of_Feat="energy"

        #chart df drop deplicate
        drop_deplicated_data=chart_df.drop_duplicates(subset=['Album Name'], keep="first").reset_index(drop=True)
        
        return drop_deplicated_data
    
    #playlist


In [517]:
spotify=SpotifyAPI(client_id,client_secret)

In [518]:
spotify.spotify_get_organised_data("titanic")["albumID"]

q=album%2Ctitanic&type=track


/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/4033701561.py:189: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/4033701561.py:189: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/4033701561.py:189: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)


0    3XwgFTxiWxep4s1VjPEaHs
1    2mgkd00677MkIIw09g0kss
Name: albumID, dtype: object

In [477]:
#playlist json 
name="spiderman"
playlists_json=spotify.search(query=name,search_type="playlist")
playlists_json


q=spiderman&type=playlist


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=spiderman&type=playlist&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': '',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/4iFrBplODblDV2Gg0zvbBl'},
    'href': 'https://api.spotify.com/v1/playlists/4iFrBplODblDV2Gg0zvbBl',
    'id': '4iFrBplODblDV2Gg0zvbBl',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706c0000bebb4a23cd64ff93df00bae5702f',
      'width': None}],
    'name': 'Spiderman - Alle Hörspiele',
    'owner': {'display_name': 'Disney Hörspiele',
     'external_urls': {'spotify': 'https://open.spotify.com/user/1rt8f3lznvh0p1ct3798qly5a'},
     'href': 'https://api.spotify.com/v1/users/1rt8f3lznvh0p1ct3798qly5a',
     'id': '1rt8f3lznvh0p1ct3798qly5a',
     'type': 'user',
     'uri': 'spotify:user:1rt8f3lznvh0p1ct3798qly5a'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'Nyw5MWQ2YzhkMWEwYTA5NWM3ZWNiOWE2YjI3ZGNlN2Y0ZTV

In [478]:
playlist_df=spotify.playlist_search_json_createdata(query=name)
playlist_df

q=spiderman&type=playlist


,item,Name,PlaylistURL,ID,ImageURL
0,0,Spiderman - Alle Hörspiele,https://open.spotify.com/playlist/4iFrBplODblD...,4iFrBplODblDV2Gg0zvbBl,https://i.scdn.co/image/ab67706c0000bebb4a23cd...
1,1,Spider-man🕷,https://open.spotify.com/playlist/1NIahjNZYEDE...,1NIahjNZYEDE9QQQNC7Vy5,https://i.scdn.co/image/ab67706c0000bebbe4eeb2...
2,2,Spider-Man: No Way Home Soundtrack,https://open.spotify.com/playlist/6nwPqanMXhuG...,6nwPqanMXhuGACQZliH4gP,https://i.scdn.co/image/ab67706c0000bebb26bac1...
3,3,spider-man vibes,https://open.spotify.com/playlist/2INWocVLeGaF...,2INWocVLeGaF5HvHoB2KFD,https://i.scdn.co/image/ab67706c0000bebbe80493...
4,4,Spiderman: Into the Spider Verse Soundtrack,https://open.spotify.com/playlist/7JDCGxFthByJ...,7JDCGxFthByJYALY0Ui5z0,https://i.scdn.co/image/ab67616d0000b273e2e352...
5,5,Miles Morales Soundtrack (Spiderman),https://open.spotify.com/playlist/7KtG0weK8vwS...,7KtG0weK8vwSjjSd5Cvokg,https://i.scdn.co/image/ab67706c0000bebb5ef5e8...
6,6,Spiderman,https://open.spotify.com/playlist/608BC68sI9NZ...,608BC68sI9NZeANHJsaWRf,https://mosaic.scdn.co/640/ab67616d0000b2733c8...
7,7,SpiderMan: No Way Home (All Songs),https://open.spotify.com/playlist/4kR1mRs8ih8n...,4kR1mRs8ih8nOU4zKwfF4Z,https://i.scdn.co/image/ab67706c0000bebb1f4615...
8,8,spider-man swinging music,https://open.spotify.com/playlist/2ub0l9XVMuNo...,2ub0l9XVMuNoAm7oN3CRwe,https://i.scdn.co/image/ab67706c0000bebbf1d6a9...
9,9,Spiderman swinging music,https://open.spotify.com/playlist/2TAVgn06KBpC...,2TAVgn06KBpCmRSinG9Ug7,https://i.scdn.co/image/ab67706c0000bebb6e93d5...


In [469]:
#get  playlist item
# https://api.spotify.com/v1/playlists/{playlist_id}/tracks
playlist_id=playlist_df["ID"][0]
# headers = spotify.get_resource_header()
# endpoint = f"https://api.spotify.com/v1/playlists/1C7xHrNHW2rjRJJkNlRKuT/tracks"
# r = requests.get(endpoint, headers=headers)
# playlist_data=r.json()
track_json=spotify.get_playlist_item_json(playlist_id
track_json

SyntaxError: invalid syntax (2640531502.py, line 9)

In [460]:
track_json["items"][2]

{'added_at': '2020-10-09T13:59:52Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1l3th6ezw269xrnq48hap37n1'},
  'href': 'https://api.spotify.com/v1/users/1l3th6ezw269xrnq48hap37n1',
  'id': '1l3th6ezw269xrnq48hap37n1',
  'type': 'user',
  'uri': 'spotify:user:1l3th6ezw269xrnq48hap37n1'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PhL2Vdao2v8SS8AptuhAr'},
     'href': 'https://api.spotify.com/v1/artists/3PhL2Vdao2v8SS8AptuhAr',
     'id': '3PhL2Vdao2v8SS8AptuhAr',
     'name': 'James Horner',
     'type': 'artist',
     'uri': 'spotify:artist:3PhL2Vdao2v8SS8AptuhAr'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',


In [461]:
for i in range(1,5):
    #URI
    lis=[]
    track=track_json["items"][i]
    lis.append(track)
lis

[{'added_at': '2020-10-09T13:59:52Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1l3th6ezw269xrnq48hap37n1'},
   'href': 'https://api.spotify.com/v1/users/1l3th6ezw269xrnq48hap37n1',
   'id': '1l3th6ezw269xrnq48hap37n1',
   'type': 'user',
   'uri': 'spotify:user:1l3th6ezw269xrnq48hap37n1'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PhL2Vdao2v8SS8AptuhAr'},
      'href': 'https://api.spotify.com/v1/artists/3PhL2Vdao2v8SS8AptuhAr',
      'id': '3PhL2Vdao2v8SS8AptuhAr',
      'name': 'James Horner',
      'type': 'artist',
      'uri': 'spotify:artist:3PhL2Vdao2v8SS8AptuhAr'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
    

In [462]:
need = []

for i in 
for i, item in enumerate(playlist_data["items"][0]["track"]):
    track = item['album']
    track_id = item['id']
    song_name = item['name']
    popularity = item['popularity']
    need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))
    
need 

SyntaxError: invalid syntax (3123787462.py, line 3)

In [36]:
#roubust
# spotify.search(query="titanic",operator="NOT",operator_query="Zone",search_type="track")

q=titanic&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=titanic&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4MCBfE4596Uoi2O4DtmEMz'},
       'href': 'https://api.spotify.com/v1/artists/4MCBfE4596Uoi2O4DtmEMz',
       'id': '4MCBfE4596Uoi2O4DtmEMz',
       'name': 'Juice WRLD',
       'type': 'artist',
       'uri': 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ

In [306]:
movie_name="titanic"
data = spotify.search({"album": f"{movie_name}"}, search_type="track")
data

q=album%2Ctitanic&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=album%2Ctitanic&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PhL2Vdao2v8SS8AptuhAr'},
       'href': 'https://api.spotify.com/v1/artists/3PhL2Vdao2v8SS8AptuhAr',
       'id': '3PhL2Vdao2v8SS8AptuhAr',
       'name': 'James Horner',
       'type': 'artist',
       'uri': 'spotify:artist:3PhL2Vdao2v8SS8AptuhAr'}],
     'available_markets': ['US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/24YoVjBTadmRW2Zhc9ou9S'},
     'href': 'https://api.spotify.com/v1/albums/24YoVjBTadmRW2Zhc9ou9S',
     'id': '24YoVjBTadmRW2Zhc9ou9S',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273065c3565b58f738e30035f10',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02065c3565b58f738e30035f10',
       'width': 300},
      {'height

In [310]:
#trackdata
need = []
for i, item in enumerate(playlist_data['tracks']['items']):
    track = item['album']
    track_id = item['id']
    song_name = item['name']
    popularity = item['popularity']
    need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity,item['album']["id"]))
    
need 

KeyError: 'tracks'

In [308]:
Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity',"albumID"))
Track_df .head(10)

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,albumID


In [201]:
access_token = spotify.access_token

headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/audio-features/"

In [202]:
Feat_df = pd.DataFrame()
for id in Track_df['Id'].iteritems():
    track_id = id[1]
    lookup_url = f"{endpoint}{track_id}"
    ra = requests.get(lookup_url, headers=headers)
    audio_feat = ra.json()
    Features_df = pd.DataFrame(audio_feat, index=[0])
    Feat_df = Feat_df.append(Features_df)
Feat_df

/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_76870/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.816,0.927,11,-2.347,1,0.2660,0.381000,0.000000,0.7740,0.5760,101.024,audio_features,6AfEWDFQxEuow52BH3JH3Y,spotify:track:6AfEWDFQxEuow52BH3JH3Y,https://api.spotify.com/v1/tracks/6AfEWDFQxEuo...,https://api.spotify.com/v1/audio-analysis/6AfE...,139800,4
0,0.643,0.632,1,-6.888,1,0.3590,0.014500,0.000000,0.1450,0.0803,155.896,audio_features,0oat0Z19USoMVuprKVlcwt,spotify:track:0oat0Z19USoMVuprKVlcwt,https://api.spotify.com/v1/tracks/0oat0Z19USoM...,https://api.spotify.com/v1/audio-analysis/0oat...,164750,4
0,0.772,0.759,0,-8.303,1,0.2410,0.138000,0.000000,0.2950,0.5280,94.029,audio_features,7HJFeZZqe5OZOEa1tISErT,spotify:track:7HJFeZZqe5OZOEa1tISErT,https://api.spotify.com/v1/tracks/7HJFeZZqe5OZ...,https://api.spotify.com/v1/audio-analysis/7HJF...,138393,4
0,0.692,0.802,1,-4.476,1,0.0715,0.000752,0.000007,0.2170,0.6610,127.973,audio_features,6xbCP59M0az3UqAMXlmhZu,spotify:track:6xbCP59M0az3UqAMXlmhZu,https://api.spotify.com/v1/tracks/6xbCP59M0az3...,https://api.spotify.com/v1/audio-analysis/6xbC...,206002,4
0,0.840,0.892,4,-6.742,1,0.1160,0.304000,0.000000,0.4410,0.4950,118.043,audio_features,3Ev7BBcXJ1ZhTMkmdkj5q7,spotify:track:3Ev7BBcXJ1ZhTMkmdkj5q7,https://api.spotify.com/v1/tracks/3Ev7BBcXJ1Zh...,https://api.spotify.com/v1/audio-analysis/3Ev7...,263830,4
0,0.646,0.536,1,-9.358,1,0.4120,0.026100,0.000000,0.1400,0.0945,155.966,audio_features,1DMn3Jk6mRXUHqKF4Bbdc2,spotify:track:1DMn3Jk6mRXUHqKF4Bbdc2,https://api.spotify.com/v1/tracks/1DMn3Jk6mRXU...,https://api.spotify.com/v1/audio-analysis/1DMn...,164676,4
0,0.426,0.870,10,-9.705,1,0.4420,0.211000,0.000000,0.1050,0.3870,82.286,audio_features,1Ed9ajrWCXoibfLvCHya8c,spotify:track:1Ed9ajrWCXoibfLvCHya8c,https://api.spotify.com/v1/tracks/1Ed9ajrWCXoi...,https://api.spotify.com/v1/audio-analysis/1Ed9...,183647,4
0,0.886,0.427,6,-10.028,1,0.1450,0.031200,0.000990,0.0906,0.2300,108.034,audio_features,5uDASfU19gDxSjW8cnCaBp,spotify:track:5uDASfU19gDxSjW8cnCaBp,https://api.spotify.com/v1/tracks/5uDASfU19gDx...,https://api.spotify.com/v1/audio-analysis/5uDA...,177184,4
0,0.697,0.622,2,-10.173,0,0.3950,0.136000,0.000000,0.4700,0.3940,114.952,audio_features,4pCG5DVtAB4Jm1KXbxtOaZ,spotify:track:4pCG5DVtAB4Jm1KXbxtOaZ,https://api.spotify.com/v1/tracks/4pCG5DVtAB4J...,https://api.spotify.com/v1/audio-analysis/4pCG...,134184,4
0,0.539,0.689,1,-8.069,0,0.3350,0.846000,0.000000,0.5200,0.7900,129.848,audio_features,3RLRQeOb49sPaosxpfWsAk,spotify:track:3RLRQeOb49sPaosxpfWsAk,https://api.spotify.com/v1/tracks/3RLRQeOb49sP...,https://api.spotify.com/v1/audio-analysis/3RLR...,244839,4


In [203]:
Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")
Full_Data 

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,albumID,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Hatesonny,KILL BILL (FAST),6AfEWDFQxEuow52BH3JH3Y,KILL BILL (FAST),2021-04-16,60,41LmucA9HuqFMDwLznWb5q,0.816,0.927,...,0.7740,0.5760,101.024,audio_features,6AfEWDFQxEuow52BH3JH3Y,spotify:track:6AfEWDFQxEuow52BH3JH3Y,https://api.spotify.com/v1/tracks/6AfEWDFQxEuo...,https://api.spotify.com/v1/audio-analysis/6AfE...,139800,4
1,1,Hatesonny,GOLDEN CHILD,0oat0Z19USoMVuprKVlcwt,KILL BILL,2020-07-24,47,0BDH7rM3aS65h8f4Vy8dto,0.643,0.632,...,0.1450,0.0803,155.896,audio_features,0oat0Z19USoMVuprKVlcwt,spotify:track:0oat0Z19USoMVuprKVlcwt,https://api.spotify.com/v1/tracks/0oat0Z19USoM...,https://api.spotify.com/v1/audio-analysis/0oat...,164750,4
2,2,Sniper Gang,Sniper Gang Presents Syko Bob & Snapkatt: Nigh...,7HJFeZZqe5OZOEa1tISErT,Halloween Bill Kill,2021-10-30,55,0kxA9Bc8pamypR5GYXmeti,0.772,0.759,...,0.2950,0.5280,94.029,audio_features,7HJFeZZqe5OZOEa1tISErT,spotify:track:7HJFeZZqe5OZOEa1tISErT,https://api.spotify.com/v1/tracks/7HJFeZZqe5OZ...,https://api.spotify.com/v1/audio-analysis/7HJF...,138393,4
3,3,Brown Eyed Girls,Black Box,6xbCP59M0az3UqAMXlmhZu,Kill Bill,2013-07-29,51,16mTBf44BkKL671OHlvfSw,0.692,0.802,...,0.2170,0.6610,127.973,audio_features,6xbCP59M0az3UqAMXlmhZu,spotify:track:6xbCP59M0az3UqAMXlmhZu,https://api.spotify.com/v1/tracks/6xbCP59M0az3...,https://api.spotify.com/v1/audio-analysis/6xbC...,206002,4
4,4,Rav,New Moon,3Ev7BBcXJ1ZhTMkmdkj5q7,Lovedrug (Off That),2019-01-21,60,7r3nD13CvCwPgbErSHLwKs,0.840,0.892,...,0.4410,0.4950,118.043,audio_features,3Ev7BBcXJ1ZhTMkmdkj5q7,spotify:track:3Ev7BBcXJ1ZhTMkmdkj5q7,https://api.spotify.com/v1/tracks/3Ev7BBcXJ1Zh...,https://api.spotify.com/v1/audio-analysis/3Ev7...,263830,4
5,5,Hatesonny,KILL BILL,1DMn3Jk6mRXUHqKF4Bbdc2,KILL BILL,2020-05-15,40,5ekVpDscyX3h6sA6wsXObA,0.646,0.536,...,0.1400,0.0945,155.966,audio_features,1DMn3Jk6mRXUHqKF4Bbdc2,spotify:track:1DMn3Jk6mRXUHqKF4Bbdc2,https://api.spotify.com/v1/tracks/1DMn3Jk6mRXU...,https://api.spotify.com/v1/audio-analysis/1DMn...,164676,4
6,6,Kill Bill: The Rapper,Ramona,1Ed9ajrWCXoibfLvCHya8c,Backwoods,2014-09-14,53,2FQKpBA2aLzkgvIV7WwazZ,0.426,0.870,...,0.1050,0.3870,82.286,audio_features,1Ed9ajrWCXoibfLvCHya8c,spotify:track:1Ed9ajrWCXoibfLvCHya8c,https://api.spotify.com/v1/tracks/1Ed9ajrWCXoi...,https://api.spotify.com/v1/audio-analysis/1Ed9...,183647,4
7,7,Rob $tone,Chill Bill (feat. J. Davi$ & Spooks),5uDASfU19gDxSjW8cnCaBp,Chill Bill (feat. J. Davi$ & Spooks),2016-06-17,80,5qBu7Ab6alstSCAzxTJMb1,0.886,0.427,...,0.0906,0.2300,108.034,audio_features,5uDASfU19gDxSjW8cnCaBp,spotify:track:5uDASfU19gDxSjW8cnCaBp,https://api.spotify.com/v1/tracks/5uDASfU19gDx...,https://api.spotify.com/v1/audio-analysis/5uDA...,177184,4
8,8,Yung Simmie,Simmie Season,4pCG5DVtAB4Jm1KXbxtOaZ,Kill Bill,2016-07-01,36,6s1m8c5SKQilSa3d00OtcO,0.697,0.622,...,0.4700,0.3940,114.952,audio_features,4pCG5DVtAB4Jm1KXbxtOaZ,spotify:track:4pCG5DVtAB4Jm1KXbxtOaZ,https://api.spotify.com/v1/tracks/4pCG5DVtAB4J...,https://api.spotify.com/v1/audio-analysis/4pCG...,134184,4
9,9,Rav,New Moon,3RLRQeOb49sPaosxpfWsAk,Dirge,2019-01-21,53,7r3nD13CvCwPgbErSHLwKs,0.539,0.689,...,0.5200,0.7900,129.848,audio_features,3RLRQeOb49sPaosxpfWsAk,spotify:track:3RLRQeOb49sPaosxpfWsAk,https://api.spotify.com/v1/tracks/3RLRQeOb49sP...,https://api.spotify.com/v1/audio-analysis/3RLR...,244839,4


In [204]:
Full_Data ["track_href"][0]

'https://api.spotify.com/v1/tracks/6AfEWDFQxEuow52BH3JH3Y'

In [205]:
Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)
Sort_DF

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,albumID,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
7,7,Rob $tone,Chill Bill (feat. J. Davi$ & Spooks),5uDASfU19gDxSjW8cnCaBp,Chill Bill (feat. J. Davi$ & Spooks),2016-06-17,80,5qBu7Ab6alstSCAzxTJMb1,0.886,0.427,...,0.0906,0.230,108.034,audio_features,5uDASfU19gDxSjW8cnCaBp,spotify:track:5uDASfU19gDxSjW8cnCaBp,https://api.spotify.com/v1/tracks/5uDASfU19gDx...,https://api.spotify.com/v1/audio-analysis/5uDA...,177184,4
11,11,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),6rhVKeUhQTGLIRmQOfc9Wo,Don't Let Me Be Misunderstood,2003-09-23,62,1FpzNqDbugk77xUj0qOrtQ,0.668,0.815,...,0.0498,0.785,117.809,audio_features,6rhVKeUhQTGLIRmQOfc9Wo,spotify:track:6rhVKeUhQTGLIRmQOfc9Wo,https://api.spotify.com/v1/tracks/6rhVKeUhQTGL...,https://api.spotify.com/v1/audio-analysis/6rhV...,628547,4
0,0,Hatesonny,KILL BILL (FAST),6AfEWDFQxEuow52BH3JH3Y,KILL BILL (FAST),2021-04-16,60,41LmucA9HuqFMDwLznWb5q,0.816,0.927,...,0.7740,0.576,101.024,audio_features,6AfEWDFQxEuow52BH3JH3Y,spotify:track:6AfEWDFQxEuow52BH3JH3Y,https://api.spotify.com/v1/tracks/6AfEWDFQxEuo...,https://api.spotify.com/v1/audio-analysis/6AfE...,139800,4
14,14,Big Boi,BOOMIVERSE,6npcJbgDhtWeNT5iOGsjZI,Kill Jill (feat. Killer Mike & Jeezy),2017-06-16,60,6bseFkqeoLaVOaPshnB34o,0.771,0.527,...,0.1070,0.114,139.935,audio_features,6npcJbgDhtWeNT5iOGsjZI,spotify:track:6npcJbgDhtWeNT5iOGsjZI,https://api.spotify.com/v1/tracks/6npcJbgDhtWe...,https://api.spotify.com/v1/audio-analysis/6npc...,264739,4
19,19,Rob $tone,"Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...",2Vo3sVSCltAaYU1u0f9ASz,"Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...",2016-08-26,60,3W3q0RCly4jC7QRlEKpyGw,0.828,0.490,...,0.0867,0.279,107.968,audio_features,2Vo3sVSCltAaYU1u0f9ASz,spotify:track:2Vo3sVSCltAaYU1u0f9ASz,https://api.spotify.com/v1/tracks/2Vo3sVSCltAa...,https://api.spotify.com/v1/audio-analysis/2Vo3...,242653,4
4,4,Rav,New Moon,3Ev7BBcXJ1ZhTMkmdkj5q7,Lovedrug (Off That),2019-01-21,60,7r3nD13CvCwPgbErSHLwKs,0.840,0.892,...,0.4410,0.495,118.043,audio_features,3Ev7BBcXJ1ZhTMkmdkj5q7,spotify:track:3Ev7BBcXJ1ZhTMkmdkj5q7,https://api.spotify.com/v1/tracks/3Ev7BBcXJ1Zh...,https://api.spotify.com/v1/audio-analysis/3Ev7...,263830,4
10,10,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),3Y9wdXMh6sNt9fUwUqTqb7,Twisted Nerve,2003-09-23,56,1FpzNqDbugk77xUj0qOrtQ,0.622,0.166,...,0.1110,0.224,115.022,audio_features,3Y9wdXMh6sNt9fUwUqTqb7,spotify:track:3Y9wdXMh6sNt9fUwUqTqb7,https://api.spotify.com/v1/tracks/3Y9wdXMh6sNt...,https://api.spotify.com/v1/audio-analysis/3Y9w...,87667,4
2,2,Sniper Gang,Sniper Gang Presents Syko Bob & Snapkatt: Nigh...,7HJFeZZqe5OZOEa1tISErT,Halloween Bill Kill,2021-10-30,55,0kxA9Bc8pamypR5GYXmeti,0.772,0.759,...,0.2950,0.528,94.029,audio_features,7HJFeZZqe5OZOEa1tISErT,spotify:track:7HJFeZZqe5OZOEa1tISErT,https://api.spotify.com/v1/tracks/7HJFeZZqe5OZ...,https://api.spotify.com/v1/audio-analysis/7HJF...,138393,4
17,17,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),6Q32Vkucx2qeuVyBd3NiFZ,Battle Without Honor Or Humanity,2003-09-23,55,1FpzNqDbugk77xUj0qOrtQ,0.704,0.844,...,0.0599,0.669,94.699,audio_features,6Q32Vkucx2qeuVyBd3NiFZ,spotify:track:6Q32Vkucx2qeuVyBd3NiFZ,https://api.spotify.com/v1/tracks/6Q32Vkucx2qe...,https://api.spotify.com/v1/audio-analysis/6Q32...,148840,4
6,6,Kill Bill: The Rapper,Ramona,1Ed9ajrWCXoibfLvCHya8c,Backwoods,2014-09-14,53,2FQKpBA2aLzkgvIV7WwazZ,0.426,0.870,...,0.1050,0.387,82.286,audio_features,1Ed9ajrWCXoibfLvCHya8c,spotify:track:1Ed9ajrWCXoibfLvCHya8c,https://api.spotify.com/v1/tracks/1Ed9ajrWCXoi...,https://api.spotify.com/v1/audio-analysis/1Ed9...,183647,4


In [206]:
chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy","albumID"]]
chart_df

,Artist,Album Name,Song Name,Release Date,Popularity,energy,albumID
7,Rob $tone,Chill Bill (feat. J. Davi$ & Spooks),Chill Bill (feat. J. Davi$ & Spooks),2016-06-17,80,0.427,5qBu7Ab6alstSCAzxTJMb1
11,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),Don't Let Me Be Misunderstood,2003-09-23,62,0.815,1FpzNqDbugk77xUj0qOrtQ
0,Hatesonny,KILL BILL (FAST),KILL BILL (FAST),2021-04-16,60,0.927,41LmucA9HuqFMDwLznWb5q
14,Big Boi,BOOMIVERSE,Kill Jill (feat. Killer Mike & Jeezy),2017-06-16,60,0.527,6bseFkqeoLaVOaPshnB34o
19,Rob $tone,"Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...","Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...",2016-08-26,60,0.490,3W3q0RCly4jC7QRlEKpyGw
4,Rav,New Moon,Lovedrug (Off That),2019-01-21,60,0.892,7r3nD13CvCwPgbErSHLwKs
10,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),Twisted Nerve,2003-09-23,56,0.166,1FpzNqDbugk77xUj0qOrtQ
2,Sniper Gang,Sniper Gang Presents Syko Bob & Snapkatt: Nigh...,Halloween Bill Kill,2021-10-30,55,0.759,0kxA9Bc8pamypR5GYXmeti
17,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),Battle Without Honor Or Humanity,2003-09-23,55,0.844,1FpzNqDbugk77xUj0qOrtQ
6,Kill Bill: The Rapper,Ramona,Backwoods,2014-09-14,53,0.870,2FQKpBA2aLzkgvIV7WwazZ


In [207]:
chart_df.drop_duplicates(subset=['Album Name'], keep="first")

,Artist,Album Name,Song Name,Release Date,Popularity,energy,albumID
7,Rob $tone,Chill Bill (feat. J. Davi$ & Spooks),Chill Bill (feat. J. Davi$ & Spooks),2016-06-17,80,0.427,5qBu7Ab6alstSCAzxTJMb1
11,Various Artists,Kill Bill Vol. 1 Original Soundtrack (PA Version),Don't Let Me Be Misunderstood,2003-09-23,62,0.815,1FpzNqDbugk77xUj0qOrtQ
0,Hatesonny,KILL BILL (FAST),KILL BILL (FAST),2021-04-16,60,0.927,41LmucA9HuqFMDwLznWb5q
14,Big Boi,BOOMIVERSE,Kill Jill (feat. Killer Mike & Jeezy),2017-06-16,60,0.527,6bseFkqeoLaVOaPshnB34o
19,Rob $tone,"Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...","Chill Bill REMIX (feat. D.R.A.M., Denzel Curry...",2016-08-26,60,0.490,3W3q0RCly4jC7QRlEKpyGw
4,Rav,New Moon,Lovedrug (Off That),2019-01-21,60,0.892,7r3nD13CvCwPgbErSHLwKs
2,Sniper Gang,Sniper Gang Presents Syko Bob & Snapkatt: Nigh...,Halloween Bill Kill,2021-10-30,55,0.759,0kxA9Bc8pamypR5GYXmeti
6,Kill Bill: The Rapper,Ramona,Backwoods,2014-09-14,53,0.870,2FQKpBA2aLzkgvIV7WwazZ


In [208]:
Name_of_Feat="energy"
st.header(f"Popularity+Energy Chart")
c = alt.Chart(chart_df).mark_circle().encode(
    alt.X('Popularity', scale=alt.Scale(zero=False)), y=f'{Name_of_Feat}', color=alt.Color('Popularity', scale=alt.Scale(type='log',scheme='rainbow')), 
    size=alt.value(300), tooltip=['Popularity', f'{Name_of_Feat}', 'Song Name', 'Album Name']).interactive()

c

alt.Chart(...)

In [197]:
st.header(f"{movie_name}sound track list")
st.table(chart_df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [198]:
#Music play feature 
drop_deplicated_data=chart_df.drop_duplicates(subset=['Album Name'], keep="first")

In [199]:
chart_df["albumID"].reset_index(drop=True)

0    5qBu7Ab6alstSCAzxTJMb1
1    1FpzNqDbugk77xUj0qOrtQ
2    41LmucA9HuqFMDwLznWb5q
3    6bseFkqeoLaVOaPshnB34o
4    3W3q0RCly4jC7QRlEKpyGw
5    7r3nD13CvCwPgbErSHLwKs
6    1FpzNqDbugk77xUj0qOrtQ
7    0kxA9Bc8pamypR5GYXmeti
8    1FpzNqDbugk77xUj0qOrtQ
9    2FQKpBA2aLzkgvIV7WwazZ
Name: albumID, dtype: object

In [ ]:
chart_df["albumID"].reset_index(drop=True)[0]

In [209]:
for i in range(len(chart_df["albumID"])):
    uri=chart_df["albumID"].reset_index(drop=True)[i]
    print(f"https://open.spotify.com/embed/album/{uri}")

https://open.spotify.com/embed/album/5qBu7Ab6alstSCAzxTJMb1
https://open.spotify.com/embed/album/1FpzNqDbugk77xUj0qOrtQ
https://open.spotify.com/embed/album/41LmucA9HuqFMDwLznWb5q
https://open.spotify.com/embed/album/6bseFkqeoLaVOaPshnB34o
https://open.spotify.com/embed/album/3W3q0RCly4jC7QRlEKpyGw
https://open.spotify.com/embed/album/7r3nD13CvCwPgbErSHLwKs
https://open.spotify.com/embed/album/1FpzNqDbugk77xUj0qOrtQ
https://open.spotify.com/embed/album/0kxA9Bc8pamypR5GYXmeti
https://open.spotify.com/embed/album/1FpzNqDbugk77xUj0qOrtQ
https://open.spotify.com/embed/album/2FQKpBA2aLzkgvIV7WwazZ


In [ ]:
drop_deplicated_data=chart_df.drop_duplicates(subset=['Album Name'], keep="first").reset_index(drop=True)
drop_deplicated_data

In [ ]:
#spotify_get_organised_data class function test
spotify_get_data=spotify.spotify_get_organised_data("titanic")
spotify_get_data

In [ ]:
!pip install streamlit

In [ ]:
#get top1
spotify_get_data["albumID"][0]

In [ ]:
import streamlit.components.v1 as components

uri=spotify_get_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )    

In [ ]:
spotify_get_data=SpotifyAPI.spotify_get_organised_data(movie_name)
spotify_get_data["albumID"][0]

uri=spotify_get_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )    

In [480]:
uri=spotify.spotify_get_organised_data["albumID"][0]
components.html(
            f"""
            <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
            allow="encrypted-media" ></iframe
                """,
                height=400
            )  

TypeError: 'method' object is not subscriptable

In [482]:
import streamlit.components.v1 as components

uri=spotify.spotify_get_organised_data["albumID"][0]
# components.html(
#             f"""
#             <iframe src=https://open.spotify.com/embed/album/{uri} width="200" height="500" frameborder="50" allowtransparency="true" 
#             allow="encrypted-media" ></iframe
#                 """,
#                 height=400
#             )  

TypeError: 'method' object is not subscriptable